<a href="https://colab.research.google.com/github/123shwetarohokale/563-ShwetaR/blob/main/project01with%20error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
from numba import njit, prange

# Function to compute autocorrelation and determine independent samples
def autocorr1D(array):
    ft = np.fft.rfft(array - np.average(array))
    acorr = np.fft.irfft(ft * np.conjugate(ft)) / (len(array) * np.var(array))
    dt = np.where(acorr < 0)[0][0]
    nsamples = len(array) // dt
    return nsamples, dt

# Lennard-Jones potential function for carbon-carbon interactions
@njit
def lennard_jones(r, epsilon=1.0, sigma=3.75):  # Methane's LJ parameters
    if r < 2.5 * sigma:  # Cutoff distance
        sr6 = (sigma / r) ** 6
        sr12 = sr6 ** 2
        return 4 * epsilon * (sr12 - sr6)
    return 0.0

# Generate a methane molecule (tetrahedral geometry)
def generate_methane(center):
    bond_length = 1.09  # Approximate C-H bond length in Angstroms
    tetrahedral_vectors = np.array([
        [1, 1, 1], [-1, -1, 1], [-1, 1, -1], [1, -1, -1]
    ]) / np.sqrt(3)
    hydrogen_positions = center + bond_length * tetrahedral_vectors
    return center, hydrogen_positions

# Monte Carlo Metropolis step
@njit(parallel=True)
def metropolis_mc(positions, L, N, beta, max_disp, n_steps):
    epsilon = 1.0
    sigma = 3.75  # Methane's sigma parameter
    E_samples = []
    acceptance = 0

    for step in range(n_steps):
        i = np.random.randint(N)
        old_pos = positions[i].copy()
        new_pos = old_pos + max_disp * (2 * np.random.rand(3) - 1)
        new_pos = np.mod(new_pos, L)  # Periodic boundary conditions

        dE = 0.0
        for j in range(N):
            if j != i:
                r_old = np.linalg.norm(old_pos - positions[j])
                r_new = np.linalg.norm(new_pos - positions[j])
                dE += lennard_jones(r_new, epsilon, sigma) - lennard_jones(r_old, epsilon, sigma)

        if dE < 0 or np.random.rand() < np.exp(-beta * dE):
            positions[i] = new_pos
            acceptance += 1

        if step % (n_steps // 100) == 0 and step > n_steps // 2:
            total_energy = sum(
                lennard_jones(np.linalg.norm(positions[i] - positions[j]), epsilon, sigma)
                for i in prange(N) for j in prange(i))
            E_samples.append(total_energy / N)

    return np.array(E_samples), acceptance / n_steps

# Simulation Parameters
T_values = [0.5, 2.0]  # Temperatures
rho_values = [0.5, 0.7]  # Densities (N/V)
N = 100  # Number of methane molecules
k_B = 1  # Boltzmann constant (dimensionless units)
n_steps = 50000  # Number of Monte Carlo steps
max_disp = 0.1  # Maximum displacement step

# Store results
results = []
performance_data = []

for T in T_values:
    beta = 1 / (k_B * T)
    for rho in rho_values:
        L = (N / rho) ** (1 / 3)  # Box length
        np.random.seed(42)
        methane_positions = L * np.random.rand(N, 3)  # Carbon atom positions

        start_time = time.time()
        E_samples, acceptance_ratio = metropolis_mc(methane_positions, L, N, beta, max_disp, n_steps)
        end_time = time.time()

        steps_per_second = n_steps / (end_time - start_time)
        performance_data.append((T, rho, steps_per_second))

        nsamples, dt = autocorr1D(E_samples)
        avg_E = np.mean(E_samples[:nsamples])
        std_E = np.std(E_samples[:nsamples])
        results.append((T, rho, avg_E, std_E, nsamples))

# Display Results
print("Simulation Results:")
for T, rho, avg_E, std_E, num_samples in results:
    print(f'T={T}, Density={rho}: <E>/N = {avg_E:.3f}, Std Dev = {std_E:.3f}, Samples = {num_samples}')

print("\nPerformance Measurements:")
for T, rho, steps_per_second in performance_data:
    print(f'T={T}, Density={rho}: {steps_per_second:.2f} steps/second')

# Summary Takeaways:
# 1. The simulation now models methane molecules with tetrahedral geometry.
# 2. Only carbon-carbon interactions are considered for Lennard-Jones potential.
# 3. Performance is improved using optimized periodic boundary conditions and Numba JIT.


UnsupportedError: Failed in nopython mode pipeline (step: inline calls to locally defined closures)
The use of yield in a closure is unsupported.

File "<ipython-input-3-6ddfa94cc08e>", line 58:
def metropolis_mc(positions, L, N, beta, max_disp, n_steps):
    <source elided>
        if step % (n_steps // 100) == 0 and step > n_steps // 2:
            total_energy = sum(
            ^
